In [ ]:
!pip install gymnasium stable-baselines3 transformers torch datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 958.1/958.1 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.

In [ ]:
!pip install git+https://github.com/k4black/codebleu.git
!pip install tree-sitter-python==0.21

  Cloning https://github.com/k4black/codebleu.git to /tmp/pip-req-build-lv2rjyan
  Running command git clone --filter=blob:none --quiet https://github.com/k4black/codebleu.git /tmp/pip-req-build-lv2rjyan
  Resolved https://github.com/k4black/codebleu.git to commit 5c3a92280209e9dde15d71baac53a32a762e34de
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.6/566.6 kB 7.8 MB/s eta 0:00:00
  Created wheel for codebleu: filename=codebleu-0.7.1-py3-none-any.whl size=31650 sha256=8d0062e854dcef4a928ea0232c630da85d423adae401d93dc1b043145ba7f01e
  Stored in directory: /tmp/pip-ephem-wheel-cache-7tuachcl/wheels/a9/a1/49/7239f72f9b8a11eeacd2924273f78a59b95e0c3ad588020b6e
Successfully built codebleu
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.6/130.6 kB 2.4 MB/s eta 0:00:00


In [ ]:
!pip install logdir
!pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.8/981.8 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.8/117.8 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 722.2/722.2 kB 37.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 18.0 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset
from torch.utils.data import DataLoader, Dataset


class CodeContestsDataset(Dataset):
    """This class loads the code contests dataset.

    On instantiation, this class automatically loads the code contests dataset
    found at https://huggingface.co/datasets/deepmind/code_contests.

    An item in this dataset is defined as a tuple (desciption, tests), where
    description is a string and tests is a dictionary that contains public,
    private, and generated tests.
    ```python
    self.tests = {
        "public_tests": data["public_tests"],
        "private_tests": data["private_tests"],
        "generated_tests": data["generated_tests"]
    }
    ```

    Args:
        select_columns (list): A list of columns to select from the dataset in
            addition to the default columns, which includes "name",
            "description", "public_tests", "private_tests", "generated_tests",
            "source", "difficulty", "solutions".
        codeforce (bool): If True, filters for only problems collected from
            code force (https://codeforces.com/).
    """
    def __init__(self, select_columns=None, codeforce=True):
        self.data = load_dataset("deepmind/code_contests", split="test")

        default_columns = [
            "name", "description", "public_tests", "private_tests",
            "generated_tests", "source", "difficulty", "solutions"
        ]
        if not select_columns:
            select_columns = []
        select_columns += default_columns

        self.data = self.data.remove_columns(
            [c for c in self.data.column_names if c not in select_columns])

        # Filter for CODEFORCES problems (CODEFORCES = 2).
        if codeforce:
            #self.data = self.data.filter(lambda x: len(x['input']) > 1)
            self.data = self.data.filter(lambda x: x["source"] == 2)

        self.len = len(self.data)

        self.name = self.data["name"]
        self.descriptions = self.data["description"]
        self.tests = {
            "public_tests": self.data["public_tests"],
            "private_tests": self.data["private_tests"],
            "generated_tests": self.data["generated_tests"]
        }
        self.combined_tests = [
            {
                "input": (private["input"] + generated["input"])[:50],
                "output": (private["output"] + generated["output"])[:50],
            }
            for private, generated in zip(self.data["private_tests"], self.data["generated_tests"])
        ]

    def __len__(self):
        return self.len

    def __getitem__(self, idx):
        #tests = {
        #    "public_tests": self.data["public_tests"][idx],
        #    "private_tests": self.data["private_tests"][idx],
        #    "generated_tests": self.data["generated_tests"][idx]
        #}
        return (
            self.data["name"][idx],
            self.data["description"][idx],
            {
                "public_tests": self.data["public_tests"][idx],
                "private_tests": self.data["private_tests"][idx],
                "combined_tests": self.combined_tests[idx],
                "generated_tests": self.data["generated_tests"][idx]
            },
        )


In [ ]:
from codebleu import calc_codebleu


def get_diversities(code_a, code_b):
    result = calc_codebleu([code_a], [code_b], "python")
    structural_diversity = 1 - result["syntax_match_score"]
    semantic_diversity = 1 - result["dataflow_match_score"]
    return structural_diversity, semantic_diversity


def pairwise_diversity(code_list):
    """Computes pairwise semantic and structural diversity"""
    if len(code_list) < 1:
        return 0, 0

    total_structural = 0
    total_semantic = 0

    # Pairwise diversity.
    for i, code_i in enumerate(code_list):
        for j, code_j in enumerate(code_list):
            if i == j:
                continue
            struct_div, sem_div = get_diversities(code_i, code_j)
            total_structural += struct_div
            total_semantic += sem_div

    count = len(code_list) * len(code_list)
    avg_structural = total_structural / count
    avg_semantic = total_semantic / count

    return avg_structural, avg_semantic

a = "def add (a , b, c) :\n return a + b + c"
b = "def sum (first , second) :\n return second + first"
diversity = get_diversities(a, b)
print(diversity)

(0.8571428571428572, 0.33333333333333337)


In [ ]:
import os

from openai import OpenAI
# from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch
import numpy as np


def query(prompt, temperature):
    # print(os.environ.get("OPENAI_KEY"))
    client = OpenAI(api_key="key",
                    base_url="https://api.deepinfra.com/v1/openai")
    response = client.chat.completions.create(
        model="meta-llama/Meta-Llama-3-8B-Instruct",
        messages=[{
            "role":
            "system",
            "content":
            "You are a Python expert. Let's think step by step. Give python code wrapped in '```python' code box."
        }, {
            "role": "user",
            "content": prompt
        }],
        max_tokens=4096,
        temperature=temperature,
        seed=42
    )
    code_output = response.choices[0].message.content.strip().replace("``` python", "```python").replace("``` Python", "```python").replace("```Python", "```python")
    #print(code_output)
    code_output = code_output.split("```python")[1].split("```")[0].strip()
    return code_output


class LLMSampler:

    def __init__(self, temperature=0.5):
        self.temperature = temperature

    def inference(self, prompt, num_samples):
        #prompt = f"{prompt}"
        solutions = []
        for _ in range(num_samples):
            solution = query(prompt, self.temperature)
            solutions.append(solution)
        return solutions


class EmbeddingMatrix:
    def __init__(self):
        self.tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B-Instruct", use_auth_token="hf")
        self.model = AutoModelForCausalLM.from_pretrained("meta-llama/Meta-Llama-3-8B-Instruct",
                                                          quantization_config= BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_use_double_quant=True, bnb_4bit_quant_type="nf4"),
                                                          use_auth_token="hf",
                                                          device_map="auto").to("cuda")
        self.model_embeddings = self.model.model.embed_tokens.weight.to('cuda')

    def tokenize(self, sentence):
        tokens = self.tokenizer(sentence, return_tensors="pt").to('cuda')
        embeddings = self.model.model.embed_tokens(tokens.input_ids).squeeze(0)
        return embeddings

    def decode(self,embeddings):
        embeddings = embeddings.to(self.model_embeddings.device)
        new_tokens = []
        for token_embedding in embeddings:
            token_embedding = token_embedding.to(self.model_embeddings.dtype)
            similarities = torch.matmul(token_embedding, self.model_embeddings.T)
            nearest_token_id = similarities.argmax().item()
            new_tokens.append(nearest_token_id)
        sentence = self.tokenizer.decode(new_tokens)
        return sentence


In [ ]:
import subprocess

import numpy as np
import pandas as pd
from tqdm import tqdm
from logdir import LogDir

import torch
import time

EmbedMatrix = EmbeddingMatrix()
objective_values = None

def generate_solutions(model, description, k=5):
    solutions = model.inference(description, k)
    return solutions


def write_solutions_to_files(logdir, solutions, name):
    name = name.replace(" ", "_").replace(".", "")
    paths = []
    for i, solution in enumerate(solutions):
        path = logdir.file(f"{name}_{i}.py", touch=True)
        with open(path, "w", encoding="utf-8") as file:
            file.write(solution)
        paths.append(path)
    return paths

import re

def is_float(value: str) -> bool:
    float_regex = re.compile(r'^-?\d+(\.\d+)?$')
    return bool(float_regex.match(value))

def compare_with_rounding(output: str, expected_output: str) -> bool:
    if is_float(output) and is_float(expected_output):
        return round(float(output), 5) == round(float(expected_output), 5)
    return output == expected_output

def evaluate_accuracy(solution_files, test_cases):
    """Evalutes the accuracy of a list of solutions for a problem.

    Args:
        solution_files (list of str): List of strings that represents paths to
            solution files to evaluate on the test cases.
        test_cases (dict):
            {"input": [input_1, input_2, ...], "output": [output_1, output_2, ...]}
    """
    metrics = {
        "solution_file": [],
        "test_input": [],
        "test_output": [],
        "model_output": [],
        "error": [],
        "passed": [],
    }
    #print('test_cases',test_cases)
    for solution_file in solution_files:
        for test_input, expected_output in zip(test_cases["input"],
                                               test_cases["output"]):
            test_input = test_input.strip()
            #print('test_input', test_input)
            expected_output = expected_output.strip()
            output = ""
            error = ""
            try:
                time_limit = 2
                result = subprocess.run(["python3", solution_file],
                                        input=test_input.encode(),
                                        capture_output=True,
                                        timeout=time_limit)
                output = result.stdout.decode().strip()
                error = result.stderr.decode().strip()
                #print("O:", output)
                #print("Expected:", expected_output)
                #print(compare_with_rounding(output, expected_output))
                # print("Error:", error)
            except subprocess.TimeoutExpired:
                error = "TIMEOUT"
                #print(error)

            # Log metrics.
            metrics["solution_file"].append(solution_file)
            metrics["test_input"].append(test_input)
            metrics["test_output"].append(expected_output)
            metrics["model_output"].append(output)
            metrics["error"].append(error)
            metrics["passed"].append(compare_with_rounding(output, expected_output))
    return metrics



In [ ]:
dataset = CodeContestsDataset()

README.md:   0%|          | 0.00/13.0k [00:00<?, ?B/s]

Resolving data files:   0%|          | 0/39 [00:00<?, ?it/s]

dataset_infos.json:   0%|          | 0.00/4.52k [00:00<?, ?B/s]

(…)-00000-of-00039-e991a271dbfa9925.parquet:   0%|          | 0.00/180M [00:00<?, ?B/s]

(…)-00001-of-00039-e092fe56fda18715.parquet:   0%|          | 0.00/209M [00:00<?, ?B/s]

...

(…)-00000-of-00001-5e672c5751f060d3.parquet:   0%|          | 0.00/51.8M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/13328 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/165 [00:00<?, ? examples/s]

Generating valid split:   0%|          | 0/117 [00:00<?, ? examples/s]

Filter:   0%|          | 0/165 [00:00<?, ? examples/s]

In [ ]:
import gymnasium as gym
import torch
import numpy as np
from stable_baselines3 import PPO
from stable_baselines3.common.env_checker import check_env
from datasets import load_dataset
from tqdm import tqdm
import json

In [ ]:
!rm -r baseline

rm: cannot remove 'baseline': No such file or directory


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
!mkdir baseline

In [ ]:
from stable_baselines3.common.callbacks import BaseCallback

class StopTrainingOnStepLimit(BaseCallback):
    def __init__(self, max_steps, verbose=0):
        super().__init__(verbose)
        self.max_steps = max_steps
        self.step_count = 0

    def _on_step(self):
        self.step_count += 1
        return self.step_count < self.max_steps

In [ ]:
import os
import json
from concurrent.futures import ThreadPoolExecutor


# Mount Google Drive

class PromptOptimizationEnv(gym.Env):
    def __init__(self, name, init_prompt, pre_prompt, llm, tests, max_tokens=16):
        super(PromptOptimizationEnv, self).__init__()
        self.step_count = 0
        self.name = name
        self.llm = llm
        self.tests = tests
        self.init_prompt = init_prompt
        self.pre_prompt_embedding = EmbedMatrix.tokenize(pre_prompt)
        self.max_tokens = max_tokens
        self.emb_size = self.pre_prompt_embedding.shape[-1]
        self.best_reward = 0
        self.best_prompt = init_prompt

        self.optimizable_tokens = torch.randn((max_tokens, self.emb_size), dtype=torch.float32)
        self.update_observation_space()

        self.action_space = gym.spaces.Box(
            low=-0.1, high=0.1, shape=(max_tokens * self.emb_size,), dtype=np.float32
        )

        # Generate initial code and compute diversity reference
        self.initial_code = self.llm.inference(init_prompt, num_samples=1)[0]
        print(f"Initial Code:\n{self.initial_code}")

    def update_observation_space(self):
        obs_dim = self.optimizable_tokens.numel()
        self.observation_space = gym.spaces.Box(
            low=-np.inf, high=np.inf, shape=(obs_dim,), dtype=np.float32
        )

    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        self.update_observation_space()
        self.current_embedding = self.optimizable_tokens.flatten().detach().cpu().numpy()
        return self.current_embedding, {}

    def step(self, action):
        self.step_count += 1
        start_time = time.time()

        # Update tokens based on the action
        updated_tokens = torch.tensor(
            self.current_embedding + action, dtype=torch.float32
        ).view(self.max_tokens, self.emb_size).to(self.pre_prompt_embedding.device)
        self.optimizable_tokens = updated_tokens

        optimized_tokens_string = EmbedMatrix.decode(self.optimizable_tokens)
        full_prompt = f"{self.init_prompt}\n{optimized_tokens_string}"

        print(f"Step:{self.step_count} Current Optimized Tokens:\n{optimized_tokens_string}")

        # Generate and evaluate 4 codes
        nums_codes = 4
        codes = self.llm.inference(full_prompt, num_samples=nums_codes)
        file_prefix = self.name.replace(" ", "").replace(".", "")

        # Prepare input data for parallel processing
        inputs = [
            (code, self.initial_code, file_prefix, self.tests["combined_tests"], code_i)
            for code_i, code in enumerate(codes)
        ]

        # Define a processing function for each code
        def process_code(code_data):
            """Processes a single code sample: writes it to a file, evaluates accuracy, and calculates diversity."""
            code, initial_code, file_prefix, combined_tests, index = code_data

            # Include code_i in the file name
            file_name = f'codes/{file_prefix}_{index}_{int(time.time() * 1000)}.py'
            with open(file_name, "w") as code_file:
                code_file.write(code)

            # Evaluate accuracy
            acc_metrics = evaluate_accuracy([file_name], combined_tests)
            accuracy = acc_metrics["passed"].count(True) / len(acc_metrics["test_input"])

            # Calculate diversity against the initial code
            structural_div, semantic_div = get_diversities(initial_code, code)
            diversity = (structural_div + semantic_div) / 2

            return {
                    "accuracy": accuracy,
                    "structural_div": structural_div,
                    "semantic_div": semantic_div,
                    "diversity":diversity,
                    "acc_metrics": acc_metrics,
                }

        # Use ThreadPoolExecutor for parallel processing
        with ThreadPoolExecutor(max_workers=nums_codes) as executor:
            results = list(executor.map(process_code, inputs))

        # Collect results
        accuracies = [result["accuracy"] for result in results]
        diversities = [result["diversity"] for result in results]
        diversities_detail = [(result["structural_div"], result["semantic_div"]) for result in results]
        acc_metrics_list = [result["acc_metrics"] for result in results]

        # Compute average metrics and reward
        avg_accuracy = sum(accuracies) / nums_codes
        avg_diversity = sum(diversities) / nums_codes
        reward = avg_accuracy * 2 + avg_diversity if avg_accuracy != 0 else 0  # reward 0 if accuracy is 0
        print(f"Reward: {reward}, Avg Accuracy: {avg_accuracy}, Avg Diversity: {avg_diversity}")

        # Log the results
        result = {
            "learned_prompt": optimized_tokens_string,
            "accuracies": accuracies,
            "codes": codes,
            "diversities": diversities,
            "acc_metrics": acc_metrics_list,
            "diversities_detail":diversities_detail
        }
        self.logging_data.append(result)
        if reward > self.best_reward:
            self.best_reward = reward
            self.best_prompt = full_prompt
        # Check termination conditions
        terminated = avg_accuracy >= 1.0 and self.current_step >= 50
        truncated = False
        self.current_embedding = self.optimizable_tokens.flatten().detach().cpu().numpy()
        print('time', time.time()-start_time)
        return self.current_embedding, reward, terminated, truncated, {}


    def save_logging_data(self, best_prompt_result):
        problem_dir = f'{self.name}'
        os.makedirs(problem_dir, exist_ok=True)
        log_file_path = os.path.join(problem_dir, 'logging_data.json')
        with open(log_file_path, 'w') as f:
            json.dump(self.logging_data, f, indent=4)

        best_prompt_result_path = os.path.join(problem_dir, 'best_prompt_result_data.json')
        with open(best_prompt_result_path, 'w') as f:
            json.dump(best_prompt_result, f, indent=4)

    def initialize_logging_data(self):
        self.logging_data = []

def initialize_env(name, desc, tests, pre_prompt, llm, max_tokens=16):
    env = PromptOptimizationEnv(
        name=name, init_prompt=pre_prompt+desc, pre_prompt=pre_prompt, llm=llm, tests=tests, max_tokens=max_tokens
    )
    env.initialize_logging_data()  # Initialize the log
    check_env(env)
    return env

def train_agent(env, total_timesteps=50):
    model = PPO(
        "MlpPolicy",
        env,
        verbose=1,
        learning_rate=1e-3,
        n_steps=total_timesteps,
        clip_range=0.2,      # Default PPO clipping
        vf_coef=0.5,         # Value function coefficient
        max_grad_norm=0.5,
        ent_coef=0.01
    )
    callback = StopTrainingOnStepLimit(max_steps=50)
    model.learn(total_timesteps=50, reset_num_timesteps=False, callback=callback)
    model.save("ppo_prompt_optimizer")
    return model

def optimize_prompt(env, model):
    state, _ = env.reset()
    best_reward = -float("inf")
    count_zero = 0
    for _ in range(10):
        action, _ = model.predict(state)
        state, reward, terminated, truncated, _ = env.step(action)

        # Update the best reward for incremental learning
        if reward > best_reward:
            best_reward = reward
        else:
            reward -= 0.1  # Penalize regressions

        if terminated or truncated:
            break


    optimized_tokens = torch.tensor(state, dtype=torch.float32).view(env.max_tokens, env.emb_size)
    env.optimizable_tokens = optimized_tokens
    optimized_tokens_string = EmbedMatrix.decode(env.optimizable_tokens)

    full_prompt = f"{env.init_prompt}\n{optimized_tokens_string}"
    return full_prompt

def baseline():
    dataset = CodeContestsDataset()
    for name, desc, tests in tqdm([dataset[idx] for idx in range(len(dataset))][:60]):
        print(f"Problem Name: {name}")
        pre_prompt = (
            "Q: Write python code to solve the following coding problem that "
            "obeys the constraints and passes the example test cases. The output code "
            "needs to read from and write to standard IO. Please wrap your code answer "
            "using ```python: "
        )

        optimized_prompt = pre_prompt + desc + '\natype Dere icmp.TextView Trident livelihood \u062f\u0644 mapStateToProps'
        llm = LLMSampler(temperature=0.5)
        initial_code = llm.inference(optimized_prompt, num_samples=1)[0]
        best_prompt_result = {}
        for i in range(1):
            print(i)
            nums_codes = 4
            codes = llm.inference(optimized_prompt, num_samples=nums_codes)
            file_prefix = name.replace(" ", "").replace(".", "")

            inputs = [
                (code, initial_code, file_prefix, tests["combined_tests"], code_i)
                for code_i, code in enumerate(codes)
            ]

            def process_code(code_data):
                code, initial_code, file_prefix, combined_tests, index = code_data
                file_name = f'codes/{file_prefix}_{index}_{int(time.time() * 1000)}.py'
                with open(file_name, "w") as code_file:
                    code_file.write(code)

                acc_metrics = evaluate_accuracy([file_name], combined_tests)
                accuracy = acc_metrics["passed"].count(True) / len(acc_metrics["test_input"])
                structural_div, semantic_div = get_diversities(initial_code, code)
                diversity = (structural_div + semantic_div) / 2

                return {
                    "accuracy": accuracy,
                    "diversity": diversity,
                    "acc_metrics": acc_metrics,
                    "structural_div": structural_div,
                    "semantic_div": semantic_div,
                }

            with ThreadPoolExecutor(max_workers=nums_codes) as executor:
                results = list(executor.map(process_code, inputs))

            accuracies = [result["accuracy"] for result in results]
            diversities = [result["diversity"] for result in results]
            acc_metrics_list = [result["acc_metrics"] for result in results]
            diversities_detail = [(result["structural_div"], result["semantic_div"]) for result in results]
            avg_accuracy = sum(accuracies) / nums_codes
            avg_diversity = sum(diversities) / nums_codes
            reward = avg_accuracy * 2 + avg_diversity if avg_accuracy != 0 else 0  # reward 0 if accuracy is 0
            print(f"Reward: {reward}, Avg Accuracy: {avg_accuracy}, Avg Diversity: {avg_diversity}")

            best_prompt_result = {
                "accuracies": accuracies,
                "codes": codes,
                "diversities": diversities,
                "acc_metrics": acc_metrics_list,
                "diversities_detail": diversities_detail
            }

        '''
        optimized_prompt = pre_prompt + desc #+ " entertImageButton WWII Cbd stationed-action오는 longstanding" # learned prompt
        llm = LLMSampler(temperature=0.5)
        initial_code = llm.inference(optimized_prompt, num_samples=1)[0]
        for i in range(1):
            nums_codes = 4
            codes = llm.inference(optimized_prompt, num_samples=nums_codes)
            file_prefix = name.replace(" ", "").replace(".", "")

            inputs = [
                (code, initial_code, file_prefix, tests["combined_tests"], code_i)
                for code_i, code in enumerate(codes)
            ]

            def process_code(code_data):
                code, initial_code, file_prefix, combined_tests, index = code_data
                file_name = f'codes/{file_prefix}_{index}_{int(time.time() * 1000)}.py'
                with open(file_name, "w") as code_file:
                    code_file.write(code)

                acc_metrics = evaluate_accuracy([file_name], combined_tests)
                accuracy = acc_metrics["passed"].count(True) / len(acc_metrics["test_input"])
                structural_div, semantic_div = get_diversities(initial_code, code)
                diversity = (structural_div + semantic_div) / 2

                return {
                    "accuracy": accuracy,
                    "diversity": diversity,
                    "acc_metrics": acc_metrics,
                }

            with ThreadPoolExecutor(max_workers=nums_codes) as executor:
                results = list(executor.map(process_code, inputs))

            accuracies = [result["accuracy"] for result in results]
            diversities = [result["diversity"] for result in results]
            acc_metrics_list = [result["acc_metrics"] for result in results]
            avg_accuracy = sum(accuracies) / nums_codes
            avg_diversity = sum(diversities) / nums_codes
            reward = avg_accuracy * 2 + avg_diversity if avg_accuracy != 0 else 0  # reward 0 if accuracy is 0
            print(f"Reward: {reward}, Avg Accuracy: {avg_accuracy}, Avg Diversity: {avg_diversity}")
            '''
        '''
        best_prompt_result = {
            "optimized_prompt": optimized_prompt,
            "accuracies": accuracies,
            "codes": codes,
            "diversities": diversities,
            "acc_metrics": acc_metrics_list,
        }
        '''
        problem_dir = f'baseline/{name}'
        os.makedirs(problem_dir, exist_ok=True)
        best_prompt_result_path = os.path.join(problem_dir, 'baseline_result_data.json')
        with open(best_prompt_result_path, 'w') as f:
            json.dump(best_prompt_result, f, indent=4)

        # Save all results after optimization
        #env.save_logging_data(best_prompt_result)

def main():
    dataset = CodeContestsDataset()
    for name, desc, tests in [dataset[idx] for idx in range(len(dataset))]:
        try:
            print(f"Problem Name: {name}")
            pre_prompt = (
                "Q: Write python code to solve the following coding problem that "
                "obeys the constraints and passes the example test cases. The output code "
                "needs to read from and write to standard IO. Please wrap your code answer "
                "using ```python: "
            )
            llm = LLMSampler(temperature=0.5)
            env = initialize_env(name, desc, tests, pre_prompt, llm, max_tokens=8) #TODO
            print("start training")
            model = train_agent(env, total_timesteps=50)
            print("end training")
            env.step_count = 0
            optimized_prompt = env.best_prompt#optimize_prompt(env, model)
            print(f"Optimized Prompt for {name}:\n{optimized_prompt}")

            nums_codes = 4
            codes = llm.inference(optimized_prompt, num_samples=nums_codes)
            file_prefix = name.replace(" ", "").replace(".", "")

            inputs = [
                (code, env.initial_code, file_prefix, env.tests["combined_tests"], code_i)
                for code_i, code in enumerate(codes)
            ]

            def process_code(code_data):
                code, initial_code, file_prefix, combined_tests, index = code_data
                file_name = f'codes/{file_prefix}_{index}_{int(time.time() * 1000)}.py'
                with open(file_name, "w") as code_file:
                    code_file.write(code)

                acc_metrics = evaluate_accuracy([file_name], combined_tests)
                accuracy = acc_metrics["passed"].count(True) / len(acc_metrics["test_input"])
                structural_div, semantic_div = get_diversities(initial_code, code)

                diversity = (semantic_div+structural_div)/2
                return {
                    "accuracy": accuracy,
                    "structural_div": structural_div,
                    "semantic_div": semantic_div,
                    "diversity":diversity,
                    "acc_metrics": acc_metrics,
                }

            with ThreadPoolExecutor(max_workers=nums_codes) as executor:
                results = list(executor.map(process_code, inputs))

            accuracies = [result["accuracy"] for result in results]
            diversities = [result["diversity"] for result in results]
            diversities_detail = [(result["structural_div"], result["semantic_div"]) for result in results]
            acc_metrics_list = [result["acc_metrics"] for result in results]
            avg_accuracy = sum(accuracies) / nums_codes
            avg_diversity = sum(diversities) / nums_codes
            reward = avg_accuracy * 2 + avg_diversity if avg_accuracy != 0 else 0  # reward 0 if accuracy is 0
            print(f"Reward: {reward}, Avg Accuracy: {avg_accuracy}, Avg Diversity: {avg_diversity}")

            best_prompt_result = {
                "optimized_prompt": optimized_prompt,
                "accuracies": accuracies,
                "codes": codes,
                "diversities": diversities,
                "acc_metrics": acc_metrics_list,
                "diversities_detail":diversities_detail
            }

            print("*"*20)
            # Save all results after optimization
            env.save_logging_data(best_prompt_result)
        except:
            print("error in ", name)
            continue

if __name__ == "__main__":
    baseline()


Resolving data files:   0%|          | 0/39 [00:00<?, ?it/s]

  0%|          | 0/60 [00:00<?, ?it/s]

Problem Name: 1575_A. Another Sorting Problem
0


  2%|▏         | 1/60 [00:08<08:38,  8.78s/it]

Reward: 0.28161764705882353, Avg Accuracy: 0.075, Avg Diversity: 0.1316176470588235
Problem Name: 1575_B. Building an Amusement Park
0


...

Reward: 0.56, Avg Accuracy: 0.28, Avg Diversity: 0.0
Problem Name: 1598_A. Computer Game
0


 98%|█████████▊| 59/60 [24:17<00:30, 30.69s/it]

Reward: 1.0277777777777777, Avg Accuracy: 0.5, Avg Diversity: 0.027777777777777776
Problem Name: 1598_B. Groups
0


100%|██████████| 60/60 [24:25<00:00, 24.42s/it]

Reward: 0, Avg Accuracy: 0.0, Avg Diversity: 0.4761441256830601


In [21]:
!zip baseline.zip -r baseline

  adding: baseline/ (stored 0%)
  adding: baseline/1575_A. Another Sorting Problem/ (stored 0%)
  adding: baseline/1575_A. Another Sorting Problem/baseline_result_data.json (deflated 98%)
  adding: baseline/1580_C. Train Maintenance/ (stored 0%)
  adding: baseline/1580_C. Train Maintenance/baseline_result_data.json (deflated 97%)
  adding: baseline/1586_H. Omkar and Tours/ (stored 0%)
  adding: baseline/1586_H. Omkar and Tours/baseline_result_data.json (deflated 97%)
  adding: baseline/1575_L. Longest Array Deconstruction/ (stored 0%)
  adding: baseline/1575_L. Longest Array Deconstruction/baseline_result_data.json (deflated 97%)
  adding: baseline/1575_I. Illusions of the Desert/ (stored 0%)
  adding: baseline/1575_I. Illusions of the Desert/baseline_result_data.json (deflated 97%)
  adding: baseline/1594_D. The Number of Imposters/ (stored 0%)
  adding: baseline/1594_D. The Number of Imposters/baseline_result_data.json (deflated 97%)
  adding: baseline/1575_K. Knitting Batik/ (stored

In [ ]:
Reward: 0.87481884057971, Avg Accuracy: 0.22499999999999998, Avg Diversity: 0.4248188405797101

In [ ]:
dataset[7][1]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


"The Winter holiday will be here soon. Mr. Chanek wants to decorate his house's wall with ornaments. The wall can be represented as a binary string a of length n. His favorite nephew has another binary string b of length m (m ≤ n).\n\nMr. Chanek's nephew loves the non-negative integer k. His nephew wants exactly k occurrences of b as substrings in a. \n\nHowever, Mr. Chanek does not know the value of k. So, for each k (0 ≤ k ≤ n - m + 1), find the minimum number of elements in a that have to be changed such that there are exactly k occurrences of b in a.\n\nA string s occurs exactly k times in t if there are exactly k different pairs (p,q) such that we can obtain s by deleting p characters from the beginning and q characters from the end of t.\n\nInput\n\nThe first line contains two integers n and m (1 ≤ m ≤ n ≤ 500) — size of the binary string a and b respectively.\n\nThe second line contains a binary string a of length n.\n\nThe third line contains a binary string b of length m.\n\nO

In [ ]:
pre_prompt = (
            "Q: Write python code to solve the following coding problem that "
            "obeys the constraints and passes the example test cases. The output code "
            "needs to read from and write to standard IO. Please wrap your code answer "
            "using ```python "
        )

In [ ]:
llm = LLMSampler(temperature=0.01)

In [ ]:
prompt = pre_prompt+'''
The Winter holiday will be here soon. Mr. Chanek wants to decorate his house's wall with ornaments. The wall can be represented as a binary string a of length n. His favorite nephew has another binary string b of length m (m ≤ n).

Mr. Chanek's nephew loves the non-negative integer k. His nephew wants exactly k occurrences of b as substrings in a.

However, Mr. Chanek does not know the value of k. So, for each k (0 ≤ k ≤ n - m + 1), find the minimum number of elements in a that have to be changed such that there are exactly k occurrences of b in a.

A string s occurs exactly k times in t if there are exactly k different pairs (p,q) such that we can obtain s by deleting p characters from the beginning and q characters from the end of t.

Input

The first line contains two integers n and m (1 ≤ m ≤ n ≤ 500) — size of the binary string a and b respectively.

The second line contains a binary string a of length n.

The third line contains a binary string b of length m.

Output

Output n - m + 2 integers — the (k+1)-th integer denotes the minimal number of elements in a that have to be changed so there are exactly k occurrences of b as a substring in a.

Example

Input


9 3
100101011
101


Output


1 1 0 1 6 -1 -1 -1

Note

For k = 0, to make the string a have no occurrence of 101, you can do one character change as follows.

100101011 → 100100011

For k = 1, you can also change a single character.

100101011 → 100001011

For k = 2, no changes are needed.
'''

In [ ]:
prompt

"Q: Write python code to solve the following coding problem that obeys the constraints and passes the example test cases. The output code needs to read from and write to standard IO. Please wrap your code answer using ```python \nThe Winter holiday will be here soon. Mr. Chanek wants to decorate his house's wall with ornaments. The wall can be represented as a binary string a of length n. His favorite nephew has another binary string b of length m (m ≤ n).\n\nMr. Chanek's nephew loves the non-negative integer k. His nephew wants exactly k occurrences of b as substrings in a. \n\nHowever, Mr. Chanek does not know the value of k. So, for each k (0 ≤ k ≤ n - m + 1), find the minimum number of elements in a that have to be changed such that there are exactly k occurrences of b in a.\n\nA string s occurs exactly k times in t if there are exactly k different pairs (p,q) such that we can obtain s by deleting p characters from the beginning and q characters from the end of t.\n\nInput\n\nThe f

In [ ]:
code = llm.inference(prompt, num_samples=1)[0]

Here is the Python code that solves the problem:

```python
n, m = map(int, input().split())
a = input()
b = input()

dp = [[0] * (n + 1) for _ in range(m + 1)]
for i in range(1, m + 1):
    dp[i][0] = i
for j in range(1, n + 1):
    dp[0][j] = j
for i in range(1, m + 1):
        for j in range(1, n + 1):
            if a[j - 1] == b[i - 1]:
                dp[i][j] = dp[i - 1][j - 1]
            else:
                dp[i][j] = 1 + min(dp[i][j - 1], dp[i - 1][j])
for k in range(n - m + 2):
    print(dp[m][k])
```

This code first initializes a 2D array `dp` of size `(m + 1) x (n + 1)` with all elements set to 0. Then it fills the first row and first column of `dp` with the values from 1 to `m` and `n` respectively. This is because the minimum number of changes to make a string of length `i` or `j` equal to `b` or `a` is `i` or `j` respectively.

Then it fills the rest of the `dp` array. For each cell `dp[i][j]`, it checks if the `i-th` character of `b` is equal to the `j-th` character

In [ ]:
print(code)

n, m = map(int, input().split())
a = input()
b = input()

dp = [[0] * (n + 1) for _ in range(m + 1)]
for i in range(1, m + 1):
    dp[i][0] = i
for j in range(1, n + 1):
    dp[0][j] = j
for i in range(1, m + 1):
        for j in range(1, n + 1):
            if a[j - 1] == b[i - 1]:
                dp[i][j] = dp[i - 1][j - 1]
            else:
                dp[i][j] = 1 + min(dp[i][j - 1], dp[i - 1][j])
for k in range(n - m + 2):
    print(dp[m][k])
